In [1]:
import sys
from pyspark import SparkConf, SparkContext
import numpy as np
import time
import re

In [2]:
# Create spark context
conf = SparkConf()
sc = SparkContext(conf=conf)


In [3]:
lines = sc.textFile('whole.txt')
n= 1000

In [4]:
lines.take(5)

['1\t2', '2\t3', '3\t4', '4\t5', '5\t6']

In [5]:
num_steps = 40
beta = 0.8
tele_const=(1-beta)/n

In [6]:
# Convert lines to pairs
lines = lines.distinct()

In [7]:
lines.take(5)

['3\t4', '6\t7', '7\t8', '8\t9', '10\t11']

In [8]:
pairsRDD = lines.map(lambda x: x.split('\t'))

In [9]:
pairsRDD.take(40)

[['3', '4'],
 ['6', '7'],
 ['7', '8'],
 ['8', '9'],
 ['10', '11'],
 ['11', '12'],
 ['18', '19'],
 ['19', '20'],
 ['21', '22'],
 ['27', '28'],
 ['28', '29'],
 ['30', '31'],
 ['35', '36'],
 ['36', '37'],
 ['37', '38'],
 ['39', '40'],
 ['42', '43'],
 ['43', '44'],
 ['45', '46'],
 ['46', '47'],
 ['47', '48'],
 ['53', '54'],
 ['56', '57'],
 ['57', '58'],
 ['59', '60'],
 ['60', '61'],
 ['61', '62'],
 ['65', '66'],
 ['66', '67'],
 ['72', '73'],
 ['74', '75'],
 ['75', '76'],
 ['80', '81'],
 ['84', '85'],
 ['87', '88'],
 ['89', '90'],
 ['90', '91'],
 ['93', '94'],
 ['94', '95'],
 ['95', '96']]

In [10]:
edgesRDD= pairsRDD.keys()

In [11]:
edgesRDD.take(5)

['3', '6', '7', '8', '10']

In [12]:
outdegRDD= edgesRDD.map(lambda x: (x,1))

In [13]:
outdegRDD.take(5)

[('3', 1), ('6', 1), ('7', 1), ('8', 1), ('10', 1)]

In [14]:
outdeg= outdegRDD.reduceByKey(lambda x,y: x+y)

In [15]:
outdeg.take(5)

[('8', 11), ('10', 5), ('19', 8), ('21', 12), ('45', 8)]

In [16]:
joinRDD= outdeg.join(pairsRDD)

In [17]:
joinRDD.take(5)

[('10', (5, '11')),
 ('10', (5, '552')),
 ('10', (5, '793')),
 ('10', (5, '287')),
 ('10', (5, '115'))]

In [18]:
#beta . M
tran_matrixRDD = joinRDD.map(lambda x: ((x[1][1],x[0]),beta/x[1][0]))


In [19]:
tran_matrixRDD.take(5)

[(('11', '10'), 0.16),
 (('552', '10'), 0.16),
 (('793', '10'), 0.16),
 (('287', '10'), 0.16),
 (('115', '10'), 0.16)]

In [20]:
# pairsRDDRev = pairsRDD.map(lambda x: (x[1],x[0]))

In [21]:
r_matrix = sc.parallelize([ [ ((str(i),str(j)),1/n) for j in range(1,n+1)] for i in range(1,n+1)]).flatMap(lambda x:x)

In [22]:
mulRDD = tran_matrixRDD.join(r_matrix)

In [23]:
mulRDD.take(5)

[(('757', '113'), (0.08888888888888889, 0.001)),
 (('9', '153'), (0.1142857142857143, 0.001)),
 (('163', '162'), (0.07272727272727274, 0.001)),
 (('924', '162'), (0.07272727272727274, 0.001)),
 (('857', '180'), (0.1, 0.001))]

In [24]:
reduce_mulRDD = mulRDD.map(lambda x: (x[0], x[1][0]*x[1][1]))

In [25]:
reduce_mulRDD.take(5)

[(('757', '113'), 8.888888888888889e-05),
 (('9', '153'), 0.0001142857142857143),
 (('163', '162'), 7.272727272727274e-05),
 (('924', '162'), 7.272727272727274e-05),
 (('857', '180'), 0.0001)]

In [26]:
reduce_keysRDD = reduce_mulRDD.map( lambda x: (x[0][0],x[1]))
reduce_keysRDD.take(5)

[('757', 8.888888888888889e-05),
 ('9', 0.0001142857142857143),
 ('163', 7.272727272727274e-05),
 ('924', 7.272727272727274e-05),
 ('857', 0.0001)]

In [27]:
ranks = reduce_keysRDD.reduceByKey( lambda x,y: x+y)

In [28]:
ranks = ranks.map(lambda x: (x[0], x[1]+tele_const))
ranks.take(5)

[('128', 0.0014834720834720836),
 ('16', 0.0016822022422022422),
 ('685', 0.000767012987012987),
 ('217', 0.0008885714285714287),
 ('815', 0.0009152958152958153)]

In [29]:
pg = ranks.collect()

In [30]:
sorted(pg,key = lambda x:x[1],reverse = True)[:5]

[('263', 0.0018759018759018761),
 ('502', 0.0018663492063492064),
 ('126', 0.001843125763125763),
 ('285', 0.001819076479076479),
 ('146', 0.0018094549894549896)]

In [31]:
sum([i[1] for i in pg] )

0.9999999999999998

#Trying out calculation... 

In [32]:
[ [ ((i,j), 1/5) for j in range(1,5+1)] for i in range(1,5+1) ]

[[((1, 1), 0.2), ((1, 2), 0.2), ((1, 3), 0.2), ((1, 4), 0.2), ((1, 5), 0.2)],
 [((2, 1), 0.2), ((2, 2), 0.2), ((2, 3), 0.2), ((2, 4), 0.2), ((2, 5), 0.2)],
 [((3, 1), 0.2), ((3, 2), 0.2), ((3, 3), 0.2), ((3, 4), 0.2), ((3, 5), 0.2)],
 [((4, 1), 0.2), ((4, 2), 0.2), ((4, 3), 0.2), ((4, 4), 0.2), ((4, 5), 0.2)],
 [((5, 1), 0.2), ((5, 2), 0.2), ((5, 3), 0.2), ((5, 4), 0.2), ((5, 5), 0.2)]]

In [33]:
pg_dict = {}
for p in pg:
    pg_dict[int(p[0])] = p[1]

In [34]:
#pg_dict[1]

Calculating page rank for 40 iteration

In [35]:
max_itr = 40
for itr in range(max_itr):
    
    #Arranging rank matrix so as to do Mapping of rank matrix and transition matrix / M matrix )
    r_matrix = sc.parallelize([ [ ((str(j),str(i)), pg_dict[i]) for j in range(1,n+1)] for i in range(1,n+1)]).flatMap(lambda x:x)
    
   
    #tran_matrixRDD=beta.M
    #corresponding  multiplication (beta.M.r)
    mulRDD = tran_matrixRDD.join(r_matrix)
    
    #corresponding addition and gettung final matrix 
    reduce_mulRDD = mulRDD.map(lambda x: (x[0], x[1][0]*x[1][1]))
    
    reduce_keysRDD = reduce_mulRDD.map( lambda x: (x[0][0],x[1]))
    ranks = reduce_keysRDD.reduceByKey( lambda x,y: x+y)
    
    #Final rank(beta.M.r + (1-beta)/N)
    ranks = ranks.map(lambda x: (x[0], x[1]+tele_const))
    pg = ranks.collect()
    for p in pg:
        pg_dict[int(p[0])] = p[1]



Top five page rank scores :

In [36]:
sorted(pg,key = lambda x:x[1],reverse = True)[:5]

[('263', 0.002020291181518219),
 ('537', 0.00194334157145315),
 ('965', 0.0019254478071662631),
 ('243', 0.0018526340162417312),
 ('285', 0.0018273721700645144)]

In [37]:
sum([i[1] for i in pg] )

0.9999999999999993

Bottom five page rank scores

In [38]:
sorted(pg,key = lambda x:x[1],reverse = False)[:5]

[('558', 0.0003286018525215297),
 ('93', 0.00035135689375165774),
 ('62', 0.00035314810510596274),
 ('424', 0.0003548153864930145),
 ('408', 0.00038779848719291705)]

In [39]:
sum([i[1] for i in pg] )

0.9999999999999993